In [ ]:
# تثبيت المكتبات
!pip install pose-format numpy

## 1️⃣ قراءة ملف .pose في Python (الطريقة الأساسية)

In [ ]:
from pose_format import Pose

def read_pose_basic(pose_path: str) -> Pose:
    """
    الطريقة الأساسية لقراءة ملف .pose
    يستخدم NumPy backend افتراضياً
    """
    with open(pose_path, 'rb') as f:
        buffer = f.read()
    
    pose = Pose.read(buffer)
    return pose

# مثال
# pose = read_pose_basic("example.pose")
# print(f"عدد الإطارات: {pose.body.data.shape[0]}")
# print(f"عدد النقاط: {pose.body.data.shape[2]}")

In [ ]:
from io import BytesIO

def read_pose_from_bytes(data: bytes) -> Pose:
    """
    قراءة من بايتات مباشرة (مفيد عند التحميل من الإنترنت)
    """
    return Pose.read(data)

def read_pose_from_bytesio(buffer: BytesIO) -> Pose:
    """
    قراءة من BytesIO (مفيد للـ streaming)
    """
    return Pose.read(buffer)

## 2️⃣ قراءة جزء محدد من الملف (للملفات الكبيرة)

In [ ]:
from io import BytesIO

def read_pose_partial(pose_path: str, 
                      start_frame: int = None, 
                      end_frame: int = None,
                      start_time: float = None,
                      end_time: float = None) -> Pose:
    """
    قراءة جزء محدد من ملف .pose
    مفيد جداً للملفات الكبيرة!
    
    Parameters:
    -----------
    pose_path : str
        مسار الملف
    start_frame : int, optional
        رقم الإطار الأول
    end_frame : int, optional
        رقم الإطار الأخير
    start_time : float, optional
        وقت البداية بالثواني
    end_time : float, optional
        وقت النهاية بالثواني
    """
    with open(pose_path, 'rb') as f:
        buffer = BytesIO(f.read())
    
    pose = Pose.read(
        buffer,
        start_frame=start_frame,
        end_frame=end_frame,
        start_time=start_time,
        end_time=end_time
    )
    
    return pose

# أمثلة:
# أول 100 إطار فقط
# pose = read_pose_partial("large_file.pose", start_frame=0, end_frame=100)

# من الثانية 5 إلى الثانية 10
# pose = read_pose_partial("large_file.pose", start_time=5.0, end_time=10.0)

## 3️⃣ قراءة Header فقط (بدون البيانات)

In [ ]:
from pose_format.pose_body import EmptyPoseBody
from io import BytesIO

def read_pose_header_only(pose_path: str):
    """
    قراءة الـ header فقط بدون تحميل البيانات
    مفيد للحصول على معلومات الملف بسرعة
    """
    with open(pose_path, 'rb') as f:
        buffer = BytesIO(f.read())
    
    pose = Pose.read(buffer, pose_body=EmptyPoseBody)
    
    print(f"معلومات الملف:")
    print(f"  الإصدار: {pose.header.version}")
    print(f"  الأبعاد: {pose.header.dimensions.width}x{pose.header.dimensions.height}")
    print(f"  عدد المكونات: {len(pose.header.components)}")
    print(f"  عدد النقاط الكلي: {pose.header.total_points()}")
    
    for comp in pose.header.components:
        print(f"    - {comp.name}: {len(comp.points)} نقطة")
    
    return pose.header

# header = read_pose_header_only("example.pose")

## 4️⃣ استخدام PyTorch Backend

In [ ]:
# تثبيت PyTorch إذا لم يكن مثبتاً
# !pip install torch

In [ ]:
def read_pose_pytorch(pose_path: str):
    """
    قراءة ملف .pose مع PyTorch backend
    مفيد للتدريب والاستدلال مع نماذج PyTorch
    """
    try:
        from pose_format.torch.pose_body import TorchPoseBody
        import torch
    except ImportError:
        print("❌ PyTorch غير مثبت. ثبته بـ: pip install torch")
        return None
    
    with open(pose_path, 'rb') as f:
        buffer = f.read()
    
    # قراءة مباشرة كـ TorchPoseBody
    pose = Pose.read(buffer, pose_body=TorchPoseBody)
    
    print(f"✅ تم التحميل كـ PyTorch tensors")
    print(f"   نوع البيانات: {type(pose.body.data)}")
    print(f"   الشكل: {pose.body.data.shape}")
    print(f"   الجهاز: {pose.body.data.device}")
    
    return pose

# pose_torch = read_pose_pytorch("example.pose")

In [ ]:
def numpy_to_pytorch(pose: Pose):
    """
    تحويل من NumPy إلى PyTorch
    """
    return pose.torch()

def move_to_gpu(pose):
    """
    نقل البيانات إلى GPU (CUDA)
    """
    if hasattr(pose.body, 'cuda'):
        pose.body = pose.body.cuda()
        print(f"✅ تم النقل إلى GPU: {pose.body.data.device}")
    return pose

# مثال:
# pose_numpy = read_pose_basic("example.pose")
# pose_torch = numpy_to_pytorch(pose_numpy)
# pose_torch = move_to_gpu(pose_torch)

## 5️⃣ استخدام TensorFlow Backend

In [ ]:
# تثبيت TensorFlow إذا لم يكن مثبتاً
# !pip install tensorflow

In [ ]:
def read_pose_tensorflow(pose_path: str):
    """
    قراءة ملف .pose مع TensorFlow backend
    """
    try:
        from pose_format.tensorflow.pose_body import TensorflowPoseBody
        import tensorflow as tf
    except ImportError:
        print("❌ TensorFlow غير مثبت. ثبته بـ: pip install tensorflow")
        return None
    
    with open(pose_path, 'rb') as f:
        buffer = f.read()
    
    pose = Pose.read(buffer, pose_body=TensorflowPoseBody)
    
    print(f"✅ تم التحميل كـ TensorFlow tensors")
    print(f"   نوع البيانات: {type(pose.body.data)}")
    print(f"   الشكل: {pose.body.data.shape}")
    
    return pose

# pose_tf = read_pose_tensorflow("example.pose")

In [ ]:
def numpy_to_tensorflow(pose: Pose):
    """
    تحويل من NumPy إلى TensorFlow
    """
    return pose.tensorflow()

# pose_numpy = read_pose_basic("example.pose")
# pose_tf = numpy_to_tensorflow(pose_numpy)

## 6️⃣ تحميل من OpenPose

In [ ]:
from pose_format.utils.openpose import load_openpose_directory

def load_from_openpose(directory: str, fps: float = 30.0, 
                       width: int = 1920, height: int = 1080) -> Pose:
    """
    تحميل بيانات OpenPose من مجلد JSON files
    
    Parameters:
    -----------
    directory : str
        مجلد يحتوي ملفات JSON من OpenPose
    fps : float
        معدل الإطارات
    width, height : int
        أبعاد الفيديو الأصلي
    """
    pose = load_openpose_directory(
        directory=directory,
        fps=fps,
        width=width,
        height=height
    )
    
    print(f"✅ تم تحميل OpenPose data")
    print(f"   الإطارات: {pose.body.data.shape[0]}")
    print(f"   المكونات: {[c.name for c in pose.header.components]}")
    
    return pose

# pose = load_from_openpose("openpose_output/", fps=30, width=1920, height=1080)

## 7️⃣ تحميل من MediaPipe

In [ ]:
from pose_format.utils.holistic import load_mediapipe_directory

def load_from_mediapipe(directory: str, fps: float = 30.0,
                        width: int = 1920, height: int = 1080) -> Pose:
    """
    تحميل بيانات MediaPipe من مجلد JSON files
    """
    pose = load_mediapipe_directory(
        directory=directory,
        fps=fps,
        width=width,
        height=height
    )
    
    print(f"✅ تم تحميل MediaPipe data")
    print(f"   الإطارات: {pose.body.data.shape[0]}")
    
    return pose

# pose = load_from_mediapipe("mediapipe_output/")

## 8️⃣ قراءة ملفات .pose في JavaScript/TypeScript

### تثبيت المكتبة
```bash
npm install pose-format
```

### استخدام في Node.js

In [ ]:
# كود JavaScript للاستخدام في Node.js
js_nodejs_code = '''
// Node.js - قراءة ملف .pose محلي
import { Pose } from 'pose-format';

async function readLocalPose(filePath: string) {
    const pose = await Pose.fromLocal(filePath);
    
    console.log('Header Info:');
    console.log(`  Version: ${pose.header.version}`);
    console.log(`  Width: ${pose.header.width}`);
    console.log(`  Height: ${pose.header.height}`);
    console.log(`  Components: ${pose.header.components.length}`);
    
    console.log('Body Info:');
    console.log(`  FPS: ${pose.body.fps}`);
    console.log(`  Frames: ${pose.body.frames.length}`);
    
    return pose;
}

// استخدام
readLocalPose('./example.pose');
'''

print("📝 كود Node.js:")
print(js_nodejs_code)

In [ ]:
# كود JavaScript للاستخدام في المتصفح
js_browser_code = '''
// Browser - قراءة ملف .pose من URL
import { Pose } from 'pose-format';

async function readRemotePose(url: string) {
    try {
        const pose = await Pose.fromRemote(url);
        
        console.log('Pose loaded successfully!');
        console.log(`Frames: ${pose.body.frames.length}`);
        console.log(`FPS: ${pose.body.fps}`);
        
        // الوصول للبيانات
        const firstFrame = pose.body.frames[0];
        const firstPerson = firstFrame.people[0];
        
        // الوصول لمكون معين (مثال: POSE_LANDMARKS)
        if (firstPerson.POSE_LANDMARKS) {
            const nose = firstPerson.POSE_LANDMARKS[0]; // أول نقطة (الأنف)
            console.log(`Nose position: X=${nose.X}, Y=${nose.Y}, Z=${nose.Z}`);
        }
        
        return pose;
    } catch (error) {
        console.error('Error loading pose:', error);
    }
}

// استخدام
readRemotePose('https://example.com/sign.pose');
'''

print("📝 كود Browser (JavaScript/TypeScript):")
print(js_browser_code)

In [ ]:
//  كود TypeScript كامل للتعامل مع pose
// ts_complete_code = '''
// TypeScript - مثال كامل
import { Pose, PoseHeaderModel, PoseBodyModel } from 'pose-format';

interface PosePoint {
    X: number;
    Y: number;
    Z: number;
    C: number;  // Confidence
}

class PoseProcessor {
    private pose: Pose;
    
    async load(source: string | Buffer): Promise<void> {
        if (typeof source === 'string') {
            if (source.startsWith('http')) {
                this.pose = await Pose.fromRemote(source);
            } else {
                this.pose = await Pose.fromLocal(source);
            }
        } else {
            this.pose = Pose.from(source);
        }
    }
    
    getFrameCount(): number {
        return this.pose.body.frames.length;
    }
    
    getFPS(): number {
        return this.pose.body.fps;
    }
    
    getComponentNames(): string[] {
        return this.pose.header.components.map(c => c.name);
    }
    
    getPointAtFrame(frameIdx: number, componentName: string, pointIdx: number): PosePoint | null {
        const frame = this.pose.body.frames[frameIdx];
        if (!frame || !frame.people[0]) return null;
        
        const component = frame.people[0][componentName];
        if (!component || !component[pointIdx]) return null;
        
        return component[pointIdx] as PosePoint;
    }
    
    // استخراج مسار نقطة عبر كل الإطارات
    getPointTrajectory(componentName: string, pointIdx: number): PosePoint[] {
        const trajectory: PosePoint[] = [];
        
        for (let i = 0; i < this.pose.body.frames.length; i++) {
            const point = this.getPointAtFrame(i, componentName, pointIdx);
            if (point) {
                trajectory.push(point);
            }
        }
        
        return trajectory;
    }
}

// استخدام
async function main() {
    const processor = new PoseProcessor();
    await processor.load('./sign_language.pose');
    
    console.log(`Frames: ${processor.getFrameCount()}`);
    console.log(`FPS: ${processor.getFPS()}`);
    console.log(`Components: ${processor.getComponentNames().join(', ')}`);
    
    // الحصول على مسار الرسغ الأيمن
    const rightWristPath = processor.getPointTrajectory('RIGHT_HAND_LANDMARKS', 0);
    console.log(`Right wrist trajectory: ${rightWristPath.length} points`);
}

main();
'''

print("📝 كود TypeScript كامل:")
print(ts_complete_code)

## 9️⃣ قراءة من URL في Python

In [ ]:
import requests

def read_pose_from_url(url: str) -> Pose:
    """
    تحميل ملف .pose من URL
    """
    response = requests.get(url)
    response.raise_for_status()
    
    pose = Pose.read(response.content)
    
    print(f"✅ تم التحميل من: {url}")
    print(f"   الإطارات: {pose.body.data.shape[0]}")
    
    return pose

# pose = read_pose_from_url("https://example.com/sign.pose")

## 🔟 دوال مساعدة شاملة

In [ ]:
class PoseReader:
    """
    كلاس شامل لقراءة ملفات Pose من مصادر متعددة
    """
    
    @staticmethod
    def from_file(path: str, backend: str = 'numpy') -> Pose:
        """
        قراءة من ملف محلي
        
        backend: 'numpy', 'torch', 'tensorflow'
        """
        with open(path, 'rb') as f:
            buffer = f.read()
        
        return PoseReader._read_with_backend(buffer, backend)
    
    @staticmethod
    def from_url(url: str, backend: str = 'numpy') -> Pose:
        """قراءة من URL"""
        import requests
        response = requests.get(url)
        response.raise_for_status()
        return PoseReader._read_with_backend(response.content, backend)
    
    @staticmethod
    def from_bytes(data: bytes, backend: str = 'numpy') -> Pose:
        """قراءة من بايتات"""
        return PoseReader._read_with_backend(data, backend)
    
    @staticmethod
    def _read_with_backend(buffer: bytes, backend: str) -> Pose:
        """قراءة مع backend محدد"""
        if backend == 'numpy':
            from pose_format.numpy import NumPyPoseBody
            return Pose.read(buffer, pose_body=NumPyPoseBody)
        
        elif backend == 'torch':
            from pose_format.torch.pose_body import TorchPoseBody
            return Pose.read(buffer, pose_body=TorchPoseBody)
        
        elif backend == 'tensorflow':
            from pose_format.tensorflow.pose_body import TensorflowPoseBody
            return Pose.read(buffer, pose_body=TensorflowPoseBody)
        
        else:
            raise ValueError(f"Backend غير معروف: {backend}")

# أمثلة:
# pose = PoseReader.from_file("example.pose", backend='numpy')
# pose = PoseReader.from_file("example.pose", backend='torch')
# pose = PoseReader.from_url("https://example.com/sign.pose")

## 📚 ملخص

### Python
| الطريقة | الاستخدام |
|---------|----------|
| `Pose.read(buffer)` | الطريقة الأساسية (NumPy) |
| `Pose.read(buffer, TorchPoseBody)` | للتدريب مع PyTorch |
| `Pose.read(buffer, TensorflowPoseBody)` | للتدريب مع TensorFlow |
| `pose.torch()` | تحويل إلى PyTorch |
| `pose.tensorflow()` | تحويل إلى TensorFlow |

### JavaScript/TypeScript
| الطريقة | الاستخدام |
|---------|----------|
| `Pose.fromLocal(path)` | قراءة ملف محلي (Node.js) |
| `Pose.fromRemote(url)` | قراءة من URL (Browser/Node.js) |
| `Pose.from(buffer)` | قراءة من Buffer |

### الخطوة التالية
انتقل للـ Notebook التالي لتعلم كيفية **عرض وتصوير الإشارات**!